# Retrieval-Augmented Generation(RAG)

## 0. Install the libraries

In [ ]:
# #langchain library
# !pip install langchain==0.1.0
# #LLM
# !pip install accelerate==0.25.0
# !pip install transformers==4.36.2
# !pip install bitsandbytes==0.41.2
# #Text Embedding
# !pip install sentence-transformers==2.2.2
# !pip install InstructorEmbedding==1.0.1
# #vectorstore to local
# !pip install pymupdf==1.23.8
# !pip install faiss-gpu==1.7.2
# !pip install faiss-cpu==1.7.4

# !pip install torch

In [2]:
# Setting up the device
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## 1. Prompt

In [3]:
from langchain import PromptTemplate

prompt_template = """
**Welcome to AIT-GPT!**  I'm your friendly AI assistant for all things AIT.

**Ask me anything related to the Asian Institute of Technology:** Whether you're curious about academic programs, research areas, faculty expertise, or campus life, I'm here to help!

**For the best results, try using specific keywords or phrases** related to your question.

**Here are some examples to get you started:**

* What are the admission requirements for international students?
* What are the current research areas at AIT?
* Can you tell me more about the faculty in the School of Engineering and Technology?
* When was AIT founded?
* What are some of the student clubs and organizations available at AIT?

**Now, ask your question!**
**Context: {context}**
**Question: {question}**

**Answer:**
    """.strip()
PROMPT=PromptTemplate.from_template(
    template=prompt_template
    )
PROMPT

PromptTemplate(input_variables=['context', 'question'], template="**Welcome to AIT-GPT!**  I'm your friendly AI assistant for all things AIT. \n\n**Ask me anything related to the Asian Institute of Technology:** Whether you're curious about academic programs, research areas, faculty expertise, or campus life, I'm here to help! \n\n**For the best results, try using specific keywords or phrases** related to your question. \n\n**Here are some examples to get you started:**\n\n* What are the admission requirements for international students?\n* What are the current research areas at AIT?\n* Can you tell me more about the faculty in the School of Engineering and Technology?\n* When was AIT founded?\n* What are some of the student clubs and organizations available at AIT?\n\n**Now, ask your question!** \n**Context: {context}**\n**Question: {question}**\n    \n**Answer:**")

In [4]:
PROMPT.format(
    context = "AIT is a leading institute in Asia for postgraduate education and applied research, focusing on engineering, technology, management, and environment.",
    question = "What are the current research areas of focus at AIT?"
)

"**Welcome to AIT-GPT!**  I'm your friendly AI assistant for all things AIT. \n\n**Ask me anything related to the Asian Institute of Technology:** Whether you're curious about academic programs, research areas, faculty expertise, or campus life, I'm here to help! \n\n**For the best results, try using specific keywords or phrases** related to your question. \n\n**Here are some examples to get you started:**\n\n* What are the admission requirements for international students?\n* What are the current research areas at AIT?\n* Can you tell me more about the faculty in the School of Engineering and Technology?\n* When was AIT founded?\n* What are some of the student clubs and organizations available at AIT?\n\n**Now, ask your question!** \n**Context: AIT is a leading institute in Asia for postgraduate education and applied research, focusing on engineering, technology, management, and environment.**\n**Question: What are the current research areas of focus at AIT?**\n    \n**Answer:**"

## 2. Retrieval

1. Document loaders : Load documents from many different sources (HTML, PDF, code).
2. Document transformers : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. Text embedding models : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. Vector stores: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. Retrievers : Once the data is in the database, you still need to retrieve it.

### 2.1 Document loaders

The document that is used has come from the Student Handbook on the [AIT Student website](https://students.ait.ac.th/)

In [5]:
from langchain.document_loaders import PyMuPDFLoader
nlp_docs = 'Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf'

loader = PyMuPDFLoader(nlp_docs)
documents =  loader.load()

In [6]:
documents[1]

Document(page_content=' \nTABLE OF CONTENTS \n \n \nI. \nIntroducing AIT \n \nII. \nStudent Bill of Rights \n \nIII. \nStudent Code of Conduct \n \nIV. \nGuidance for New Students \n \nV. \nStudent Welfare Unit and the AIT Career Center \n \nVI. \nHarassment Policy \n \nVII. \nSubstance Abuse Policy \n \nVIII. \nEnvironment Policy \n \nIX. \nStudent Organizations and Student Participation in Institute Governance \n- Student Union \n- Nationality Associations \n- Student Participation in Institute Governance \n \nX. \nCampus Facilities and Services \n- Student Accommodation \n- Visas \n- Banking \n- Dining \n- Sports and Recreation \n- Movies on Campus \n- Religious Services \n- Mails \n- AIT Reception \n- Office of Public Affairs (OPA) \n- Books and Office Supplies \n- English Language Training and Advising \n- Library \n- Internet and Other Campus Wide IT Services \n- Child Schooling \n- Medical Examination and Emergency \n- Security Service \n \nXI. \nGetting Around \n- On Campus \n-

In [7]:
len(documents) #document per page

95

To prevent hallucination for receiving unnecessary data, we need to brake documents as a chunk.

### 2.2 Document Transformers

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap=100
)
doc = text_splitter.split_documents(documents)

In [9]:
doc[1]

Document(page_content='TABLE OF CONTENTS \n \n \nI. \nIntroducing AIT \n \nII. \nStudent Bill of Rights \n \nIII. \nStudent Code of Conduct \n \nIV. \nGuidance for New Students \n \nV. \nStudent Welfare Unit and the AIT Career Center \n \nVI. \nHarassment Policy \n \nVII. \nSubstance Abuse Policy \n \nVIII. \nEnvironment Policy \n \nIX. \nStudent Organizations and Student Participation in Institute Governance \n- Student Union \n- Nationality Associations \n- Student Participation in Institute Governance \n \nX. \nCampus Facilities and Services \n- Student Accommodation \n- Visas \n- Banking \n- Dining \n- Sports and Recreation \n- Movies on Campus \n- Religious Services \n- Mails \n- AIT Reception \n- Office of Public Affairs (OPA)', metadata={'source': 'Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'file_path': 'Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'page': 1, 'total_pages': 95, 'format': 'PDF 1.4', 'title': '(\\(NDIJobGUID=fb11135f-6

In [10]:
len(doc)

371

### 2.3 Text Embedding Models

In [11]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


load INSTRUCTOR_Transformer


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


max_seq_length  512


### 2.4 Vector Stores

In [12]:
import os
#locate the vector store
vector_path = './vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print("Created the vector-store file")

In [13]:
#save vector locally
from langchain.vectorstores import FAISS
vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)
vectordb.save_local(
    folder_path=vector_path,
    index_name = 'nlp'#default index

)

### 2.5 Retrivers

In [14]:
vectordb = FAISS.load_local(
    folder_path = vector_path,
    embeddings= embedding_model,
    index_name = "nlp"
)

In [15]:
# Ready to use
retriever = vectordb.as_retriever()

In [16]:
retriever.get_relevant_documents("What is SET")

[Document(page_content='Services at AIT or unlawful use of UIA, will be penalized ranging from blocking Internet \naccess for a period of various length depending on its severity and frequency up to \nexpulsion from the Institute. \n \nThe Institute has implemented computerized systems for almost all Information \nSystem applications. Student and academic records are available online from Student \nInformation System (SIS) accessible from https://sis.ait.ac.th for SET and SERD \nstudents, https://somsis.ait.ac.th for SOM’s students. Almost all actions related to \nstudent academic records are performed online through SIS/SOMSIS ranging from', metadata={'source': 'Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'file_path': 'Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'page': 79, 'total_pages': 95, 'format': 'PDF 1.4', 'title': '(\\(NDIJobGUID=fb11135f-67cd-4d80-beec-458427a81384\\)Microsoft Word - Student Handbook_August 2023 Semester_FINAL_as 

## 3. Memory

In [17]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [18]:
history.add_user_message('Hi')
history.add_ai_message('What\'s up?')

In [19]:
history

ChatMessageHistory(messages=[HumanMessage(content='Hi'), AIMessage(content="What's up?")])

#### Conversation buffer

In [20]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'},{'output':'What\'s up?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?"}

In [21]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({'input':'hi'},{'output':'What\'s up?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'), AIMessage(content="What's up?")]}

#### Conversation buffer window (limit the number of history)

In [22]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)
memory.save_context({'input':'hi'},{'output':'What\'s up?'})
memory.save_context({'input':'What???'},{'output':'I dont know!'})
memory.save_context({'input':'haha'},{'output':'up?'})
memory.load_memory_variables({})

{'history': 'Human: What???\nAI: I dont know!\nHuman: haha\nAI: up?'}

## 4. Chain

In [23]:
# %cd ./models
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0
# %cd ..
# kinda too big for webapplication

[Errno 2] No such file or directory: './models'
/content
fatal: destination path 'fastchat-t5-3b-v1.0' already exists and is not an empty directory.
/


In [26]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline

model_id = './fastchat-t5-3b-v1.0/'

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config=bitsandbyte_config,  # caution Nvidia
    device_map='auto',
    load_in_8bit=True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    batch_size=1,
    model_kwargs={
        "temperature": 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline=pipe)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Class ConversationalRetrievalChain

In [28]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [29]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [30]:
# integrate LLM and Prompt
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [31]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  is  the  difference  between  Machine  Learning  and  Deep  Learning  AI?\n'}

combine_docs_chain

In [32]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="**Welcome to AIT-GPT!**  I'm your friendly AI assistant for all things AIT. \n\n**Ask me anything related to the Asian Institute of Technology:** Whether you're curious about academic programs, research areas, faculty expertise, or campus life, I'm here to help! \n\n**For the best results, try using specific keywords or phrases** related to your question. \n\n**Here are some examples to get you started:**\n\n* What are the admission requirements for international students?\n* What are the current research areas at AIT?\n* Can you tell me more about the faculty in the School of Engineering and Technology?\n* When was AIT founded?\n* What are some of the student clubs and organizations available at AIT?\n\n**Now, ask your question!** \n**Context: {context}**\n**Question: {question}**\n    \n**Answer:**"), llm=HuggingFacePipeline(pipeline=<transformer

In [33]:
query = "What is AIT?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
**Welcome to AIT-GPT!**  I'm your friendly AI assistant for all things AIT. 

**Ask me anything related to the Asian Institute of Technology:** Whether you're curious about academic programs, research areas, faculty expertise, or campus life, I'm here to help! 

**For the best results, try using specific keywords or phrases** related to your question. 

**Here are some examples to get you started:**

* What are the admission requirements for international students?
* What are the current research areas at AIT?
* Can you tell me more about the faculty in the School of Engineering and Technology?
* When was AIT founded?
* What are some of the student clubs and organizations available at AIT?

**Now, ask your question!** 
**Context: a self-contained international community with a cosmopolitan approach to living and 
learning. 
 
Since 1959, AIT has carried out its mission “to develop 

{'input_documents': [Document(page_content='a self-contained international community with a cosmopolitan approach to living and \nlearning. \n \nSince 1959, AIT has carried out its mission “to develop highly qualified and committed \nprofessionals who play a leading role in the region’s sustainable development and its \nintegration into the global economy” by supporting technological change and \nsustainable development through higher learning, research, capacity building and \noutreach. \n \nAIT’s renowned degree programs are administered by its School of Engineering and \nTechnology; School of Environment, Resources and Development; and School of \nManagement. Students benefit from challenging academic programs and exciting', metadata={'source': 'Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'file_path': 'Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'page': 2, 'total_pages': 95, 'format': 'PDF 1.4', 'title': '(\\(NDIJobGUID=fb11135f-67cd-4d8

In [34]:
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="**Welcome to AIT-GPT!**  I'm your friendly AI assistant for all things AIT. \n\n**Ask me anything related to the Asian Institute of Technology:** Whether you're curious about academic programs, research areas, faculty expertise, or campus life, I'm here to help! \n\n**For the best results, try using specific keywords or phrases** related to your question. \n\n**Here are some examples to get you started:**\n\n* What are the admission requirements for international students?\n* What are the current research areas at AIT?\n* Can you tell me more about the faculty in the School of Engineering and Technology?\n* When was AIT founded?\n* What are some of the student clubs and organ

## 5. Chatbot

In [35]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
**Welcome to AIT-GPT!**  I'm your friendly AI assistant for all things AIT. 

**Ask me anything related to the Asian Institute of Technology:** Whether you're curious about academic programs, research areas, faculty expertise, or campus life, I'm here to help! 

**For the best results, try using specific keywords or phrases** related to your question. 

**Here are some examples to get you started:**

* What are the admission requirements for international students?
* What are the current research areas at AIT?
* Can you tell me more about the faculty in the School of Engineering and Technology?
* When was AIT founded?
* What are some of the student clubs and organizations available at AIT?

**Now, ask your question!** 
**Context: involved in the claimed harassment, abuse or discrimination, and acceptable to 
both sides, (probab

{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': '<pad> I  am  an  AI  assistant  created  by  Google  that  can  assist  you  with  information  about  the  Asian  Institute  of  Technology  (AIT).  I  am  designed  to  provide  helpful  and  informative  responses  to  your  questions  about  AIT.  How  can  I  assist  you  today?\n',
 'source_documents': [Document(page_content='involved in the claimed harassment, abuse or discrimination, and acceptable to \nboth sides, (probably, though not exclusively, someone from the list in Section \nC-2.3 above). The agreed individual(s) will meet privately and/or together with \neach of the sides involved, attempting to clarify their perceptions, and to reach \na mutually- acceptable understanding for their future interactions. Outcomes of', metadata={'source': 'Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'file_path': 'Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'page': 38, '

In [36]:
prompt_question = "How many Academic Program?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad> I  am  an  AI  assistant  created  by  Google  that  can  assist  you  with  information  about  the  Asian  Institute  of  Technology  (AIT).  I  am  designed  to  provide  helpful  and  informative  responses  to  your  questions  about  AIT.  How  can  I  assist  you  today?\n')]
Follow Up Input: How many Academic Program?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
**Welcome to AIT-GPT!**  I'm your friendly AI assistant for all things AIT. 

**Ask me anything related to the Asian Institute of Technology:** Whether you're curious about ac

{'question': 'How many Academic Program?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad> I  am  an  AI  assistant  created  by  Google  that  can  assist  you  with  information  about  the  Asian  Institute  of  Technology  (AIT).  I  am  designed  to  provide  helpful  and  informative  responses  to  your  questions  about  AIT.  How  can  I  assist  you  today?\n')],
 'answer': '<pad> AIT  offers  a  total  of  62  academic  programs,  including  undergraduate,  graduate,  and  postgraduate  degrees.\n',
 'source_documents': [Document(page_content='a self-contained international community with a cosmopolitan approach to living and \nlearning. \n \nSince 1959, AIT has carried out its mission “to develop highly qualified and committed \nprofessionals who play a leading role in the region’s sustainable development and its \nintegration into the global economy” by supporting technological change and \nsustainable development through highe

In [37]:
prompt_question = "How many school in AIT?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad> I  am  an  AI  assistant  created  by  Google  that  can  assist  you  with  information  about  the  Asian  Institute  of  Technology  (AIT).  I  am  designed  to  provide  helpful  and  informative  responses  to  your  questions  about  AIT.  How  can  I  assist  you  today?\n'), HumanMessage(content='How many Academic Program?'), AIMessage(content='<pad> AIT  offers  a  total  of  62  academic  programs,  including  undergraduate,  graduate,  and  postgraduate  degrees.\n')]
Follow Up Input: How many school in AIT?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Pro

{'question': 'How many school in AIT?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad> I  am  an  AI  assistant  created  by  Google  that  can  assist  you  with  information  about  the  Asian  Institute  of  Technology  (AIT).  I  am  designed  to  provide  helpful  and  informative  responses  to  your  questions  about  AIT.  How  can  I  assist  you  today?\n'),
  HumanMessage(content='How many Academic Program?'),
  AIMessage(content='<pad> AIT  offers  a  total  of  62  academic  programs,  including  undergraduate,  graduate,  and  postgraduate  degrees.\n')],
 'answer': '<pad> There  are  three  schools  at  AIT:  School  of  Engineering  and  Technology,  School  of  Environment,  Resources  and  Development,  and  School  of  Management.\n',
 'source_documents': [Document(page_content='a self-contained international community with a cosmopolitan approach to living and \nlearning. \n \nSince 1959, AIT has carried out its mission

## 6. Test model performance

### General Retrieval Performance

In [38]:
# Factual Questions 1
prompt_question = "Who is the current president of AIT?"
answer = chain({"question":prompt_question})
answer["answer"]



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad> I  am  an  AI  assistant  created  by  Google  that  can  assist  you  with  information  about  the  Asian  Institute  of  Technology  (AIT).  I  am  designed  to  provide  helpful  and  informative  responses  to  your  questions  about  AIT.  How  can  I  assist  you  today?\n'), HumanMessage(content='How many Academic Program?'), AIMessage(content='<pad> AIT  offers  a  total  of  62  academic  programs,  including  undergraduate,  graduate,  and  postgraduate  degrees.\n'), HumanMessage(content='How many school in AIT?'), AIMessage(content='<pad> There  are  three  schools  at  AIT:  School  of  Engineering  and  Technology,  School  of

'<pad> The  current  president  of  AIT  is  Dr.  Suthep  Chaiyapong.\n'

In [39]:
# Factual Questions 2
prompt_question = "What is the year AIT was founded?"
answer = chain({"question":prompt_question})
answer["answer"]



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='How many Academic Program?'), AIMessage(content='<pad> AIT  offers  a  total  of  62  academic  programs,  including  undergraduate,  graduate,  and  postgraduate  degrees.\n'), HumanMessage(content='How many school in AIT?'), AIMessage(content='<pad> There  are  three  schools  at  AIT:  School  of  Engineering  and  Technology,  School  of  Environment,  Resources  and  Development,  and  School  of  Management.\n'), HumanMessage(content='Who is the current president of AIT?'), AIMessage(content='<pad> The  current  president  of  AIT  is  Dr.  Suthep  Chaiyapong.\n')]
Follow Up Input: What is the year AIT was founded?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsC

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


'<pad>  1959\n'

In [40]:
#Open Ended Questions
prompt_question = "How does AIT contribute to sustainable development in the region?"
answer = chain({"question":prompt_question})
answer["answer"]



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='How many school in AIT?'), AIMessage(content='<pad> There  are  three  schools  at  AIT:  School  of  Engineering  and  Technology,  School  of  Environment,  Resources  and  Development,  and  School  of  Management.\n'), HumanMessage(content='Who is the current president of AIT?'), AIMessage(content='<pad> The  current  president  of  AIT  is  Dr.  Suthep  Chaiyapong.\n'), HumanMessage(content='What is the year AIT was founded?'), AIMessage(content='<pad>  1959\n')]
Follow Up Input: How does AIT contribute to sustainable development in the region?
Standalone question:


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
**Welcome to AIT-GPT!**  I'm your friendly AI assistant for all things AIT. 

**Ask me anything related to the Asian Institute of Technology:** Whether you're curious about academic programs, research areas, faculty expertise, or campus life, I'm here to help! 

**For the best results, try using specific keywords or phrases** related to your question. 

**Here are some examples to get you started:**

* What are the admission requirements for international students?
* What are the current research areas at AIT?
* Can you tell me more about the faculty in the School of Engineering and Technology?
* When was AIT founded?
* What are some of the student clubs and organizations available at AIT?

**Now, ask your question!** 
**Context: a self-contained international community with a cosmopolitan approach to living and 
learning. 
 
Since 1959, AIT has carried out its m

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


'<pad> AIT  contributes  to  sustainable  development  in  the  region  through  its  mission  of  "to  develop  highly  qualified  and  committed  professionals  who  play  a  leading  role  in  the  region’s  sustainable  development  and  its  integration  into  the  global  economy"  by  supporting  technological  change  and  sustainable  development  through  higher  learning,  research,  capacity  building,  and  outreach.  AIT\'s  renowned  degree  programs  are  administered  by  its  School  of  Engineering  and  Technology;  School  of  Environment,  Resources  and  Development;  and  School  of  Management.  Students  benefit  from  challenging  academic  programs  and  exciting  research  opportunities  in  fields  such  as  renewable  energy,  sustainable  transportation,  and  sustainable  management.  AIT  also  provides  outreach  and  engagement  programs  to  support  sustainable  development  in  the  region,  such  as  capacity  building  workshops  and  seminars,'

### Real Example Analysis with AIT-GPT Prompts

In [41]:
# Retrieval Performance
prompt_question = "What are the admission requirements for international students at AIT?"
answer = chain({"question":prompt_question})
answer['answer']



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who is the current president of AIT?'), AIMessage(content='<pad> The  current  president  of  AIT  is  Dr.  Suthep  Chaiyapong.\n'), HumanMessage(content='What is the year AIT was founded?'), AIMessage(content='<pad>  1959\n'), HumanMessage(content='How does AIT contribute to sustainable development in the region?'), AIMessage(content='<pad> AIT  contributes  to  sustainable  development  in  the  region  through  its  mission  of  "to  develop  highly  qualified  and  committed  professionals  who  play  a  leading  role  in  the  region’s  sustainable  development  and  its  integration  into  the  global  economy"  by  supporting  technological  change  and  sustainable  development  throu

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
**Welcome to AIT-GPT!**  I'm your friendly AI assistant for all things AIT. 

**Ask me anything related to the Asian Institute of Technology:** Whether you're curious about academic programs, research areas, faculty expertise, or campus life, I'm here to help! 

**For the best results, try using specific keywords or phrases** related to your question. 

**Here are some examples to get you started:**

* What are the admission requirements for international students?
* What are the current research areas at AIT?
* Can you tell me more about the faculty in the School of Engineering and Technology?
* When was AIT founded?
* What are some of the student clubs and organizations available at AIT?

**Now, ask your question!** 
**Context: studies by providing testing services, one-on-one tutorials, language 
programs, academic writing courses and workshops. 
 
English is 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


'<pad> The  admission  requirements  for  international  students  at  AIT  include:\n 1.  English  proficiency:  All  degree  program  students  are  required  to  take  the  AIT  English  Entry  Test  (EET)  or  have  a  certificate  of  English  Medium  Instruction  before  they  can  graduate.\n 2.  High  school  diploma  or  equivalent:  International  students  must  have  a  high  school  diploma  or  equivalent  from  a  recognized  institution  in  their  country.\n 3.  English  proficiency:  All  degree  program  students  are  required  to  take  the  AIT  Writing  Test  (AIT-WT)  upon  arrival  on  campus.\n 4.  Proof  of  a  self-contained  international  community:  International  students  must  provide  proof  of  a  self-contained  international  community  with  a  cosmopolitan  approach  to  living  and  learning.\n 5.  Proof  of  financial '

In [42]:
# Specificity and Relevance
prompt_question = "That's a good start. Can you provide some specific details about the English language proficiency requirements, such as the minimum score needed on the TOEFL or IELTS exams?"
answer = chain({"question":prompt_question})
answer['answer']



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='What is the year AIT was founded?'), AIMessage(content='<pad>  1959\n'), HumanMessage(content='How does AIT contribute to sustainable development in the region?'), AIMessage(content='<pad> AIT  contributes  to  sustainable  development  in  the  region  through  its  mission  of  "to  develop  highly  qualified  and  committed  professionals  who  play  a  leading  role  in  the  region’s  sustainable  development  and  its  integration  into  the  global  economy"  by  supporting  technological  change  and  sustainable  development  through  higher  learning,  research,  capacity  building,  and  outreach.  AIT\'s  renowned  degree  programs  are  administered  by  its  School  of  Engineer

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
**Welcome to AIT-GPT!**  I'm your friendly AI assistant for all things AIT. 

**Ask me anything related to the Asian Institute of Technology:** Whether you're curious about academic programs, research areas, faculty expertise, or campus life, I'm here to help! 

**For the best results, try using specific keywords or phrases** related to your question. 

**Here are some examples to get you started:**

* What are the admission requirements for international students?
* What are the current research areas at AIT?
* Can you tell me more about the faculty in the School of Engineering and Technology?
* When was AIT founded?
* What are some of the student clubs and organizations available at AIT?

**Now, ask your question!** 
**Context: studies by providing testing services, one-on-one tutorials, language 
programs, academic writing courses and workshops. 
 
English is 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


"<pad> The  minimum  score  required  on  the  TOEFL  or  IELTS  exams  for  international  students  at  AIT  is  6.0  on  the  TOEFL  or  6.5  on  the  IELTS.  However,  it's  important  to  note  that  the  specific  requirements  may  vary  depending  on  the  program  and  the  specific  requirements  of  the  program.\n **Context:  The  minimum  score  required  on  the  TOEFL  or  IELTS  exams  for  international  students  at  AIT  is  6.0  on  the  TOEFL  or  6.5  on  the  IELTS.  However,  it's  important  to  note  that  the  specific  requirements  may  vary  depending  on  the  program  and  the  specific  requirements  of  the  program.\n **As  an  AI  assistant,  I  don't  have  access  to  the  most  up-to-"

In [43]:
# Error Analysis
prompt_question = "The information about scholarships is helpful, but can you focus on the question about admission requirements first?  For instance, are there any additional documents needed besides transcripts?"
answer = chain({"question":prompt_question})
answer['answer']



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='How does AIT contribute to sustainable development in the region?'), AIMessage(content='<pad> AIT  contributes  to  sustainable  development  in  the  region  through  its  mission  of  "to  develop  highly  qualified  and  committed  professionals  who  play  a  leading  role  in  the  region’s  sustainable  development  and  its  integration  into  the  global  economy"  by  supporting  technological  change  and  sustainable  development  through  higher  learning,  research,  capacity  building,  and  outreach.  AIT\'s  renowned  degree  programs  are  administered  by  its  School  of  Engineering  and  Technology;  School  of  Environment,  Resources  and  Development;  and  School  of 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
**Welcome to AIT-GPT!**  I'm your friendly AI assistant for all things AIT. 

**Ask me anything related to the Asian Institute of Technology:** Whether you're curious about academic programs, research areas, faculty expertise, or campus life, I'm here to help! 

**For the best results, try using specific keywords or phrases** related to your question. 

**Here are some examples to get you started:**

* What are the admission requirements for international students?
* What are the current research areas at AIT?
* Can you tell me more about the faculty in the School of Engineering and Technology?
* When was AIT founded?
* What are some of the student clubs and organizations available at AIT?

**Now, ask your question!** 
**Context: For non-Thai students, four additional photos must be submitted to the Government 
Relations Unit for a visa. Please write your full na

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


'<pad> Yes,  in  addition  to  transcripts,  international  students  at  AIT  are  required  to  submit  four  additional  photos  for  a  visa.  Please  write  your  full  name,  AIT  School,  and  Nationality  at  the  back  of  all  photographs.\n'

## AIT-GPT Model Performance Analysis

### 1. Information Retrieval
The model demonstrates potential in retrieving information related to AIT, with strengths and weaknesses depending on the question type:

**Strengths:**
- Successfully retrieved relevant details about admission requirements for international students.
- Provided specific details when prompted for clarification (e.g., minimum TOEFL/IELTS scores).
- Retrieved factual information for questions about the year AIT was founded.

**Weaknesses:**
- May include redundant information within the response (e.g., duplicate points on English proficiency).
- Might struggle with complex prompts requiring disambiguation (e.g., misinterpreting "additional documents" for admission).
- May retrieve outdated information for non-data-provided questions (e.g., current AIT president).
- Responses to open-ended questions could benefit from improved comprehensiveness and specificity.

### 2. Unrelated Information
The model occasionally provides information unrelated to the user's query, as seen in Error Analysis where it focused on visa photos instead of additional admission documents. And also the initial response for the non-data provided question regarding the current AIT president might be outdated. This highlights the limitations of the model in handling frequently changing information.


### Overall
The AIT-GPT model shows promise in retrieving factual information about AIT. Addressing the identified issues through prompt improvements, data augmentation, and ensuring access to current information can significantly enhance its accuracy and user experience.